<a href="https://colab.research.google.com/github/ililililililililililililililil/INI/blob/ini_pytorch/ini_5_1_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# Training Data

x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [4]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [5]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [6]:
# Computing the Hypothesis (1)

print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

e^1 equals:  tensor([2.7183])


In [7]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [8]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b))) #x_train.matmul(W): X*W

In [9]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


In [10]:
# Computing the Hypothesis (2)

print('1/(1+e^{-1}) equals: ', torch.sigmoid(torch.FloatTensor([1])))

1/(1+e^{-1}) equals:  tensor([0.7311])


In [11]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

In [12]:
print(hypothesis)
print(hypothesis.shape)

#sigmoid 사용: Computing the Hypothesis (1)과 같은 결과

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
torch.Size([6, 1])


In [13]:
# Computing the Cost Function

print(hypothesis)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


In [14]:
-(y_train[0] * torch.log(hypothesis[0]) + 
 (1 - y_train[0]) * torch.log(1 - hypothesis[0])) #element 하나

tensor([0.6931], grad_fn=<NegBackward0>)

In [15]:
losses = -(y_train * torch.log(hypothesis) + 
 (1 - y_train) * torch.log(1 - hypothesis)) #전체
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward0>)


In [16]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [17]:
F.binary_cross_entropy(hypothesis, y_train) #이전까지를 한번에

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

In [18]:
#Whole Training Procedure

# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

  # Cost 계산
  hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
  cost = F.binary_cross_entropy(hypothesis, y_train)

  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031672
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


In [19]:
# Evaluation

hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis[:5])
#강의자료랑 결과가 나온다

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01]], grad_fn=<SliceBackward0>)


In [20]:
prediction = hypothesis >= torch.FloatTensor([0.5]) #0.5 이상이면 True
print(prediction[:5])
#강의자료랑 다른 결과가 나온다. 강의자료는 0 1 0 1 0 으로 T/F도 아니고 에측도 다름
#예측이 다른 이유는 그냥 강의자료 정답이랑 내 정답이랑 달라서 생긴 문제였다

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])


In [21]:
print(prediction[:5]) #예측
print(y_train[:5]) #정답
#강의자료랑 다르지만 예측은 맞음

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])


In [22]:
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])
#정답 (5개에 대해)

tensor([[True],
        [True],
        [True],
        [True],
        [True]])


In [23]:
#Higher Implementation with Class

#실전

class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [24]:
model = BinaryClassifier()

In [26]:
# # optimizer 설정
# optimizer = optim.SGD(model.parameters(), lr=1)

# nb_epochs = 100
# for epoch in range(nb_epochs + 1):

#   # H(x) 계산
#   hypothesis = model(x_train)

#   # Cost 계산
#   cost = F.binary_cross_entropy(hypothesis, y_train)

#   # cost로 H(x) 개선
#   optimizer.zero_grad()
#   cost.backward()
#   optimizer.step()

#   # 20번마다 로그 출력
#   if epoch % 10 == 0:
#     prediction = hypothesis >= torch.FloatTensor([0.5])
#     correct_prediction = prediction.float() == y_train
#     accuracy = correct_prediction.sum().item() / len(correct_prediction)
#     print('Epoch {:4d}/{} Cost: {:6f} Accuracy {:2.2f}%'.format(
#         epoch, nb_epochs, cost.item(), accuracy * 100,
#     ))

#에러: self.linear = nn.Linear(8, 1)에서 크기가 현재 x_train과 맞지 않아 계산이 안 되는 것으로 보인다